<a href="https://colab.research.google.com/github/Mr-Ye-Cao/Rice_Visual_University_Visual_Class/blob/main/Quick_Thought_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup

URL = 'https://courses.rice.edu/courses/!SWKSCAT.cat?p_acyr_code=2021&p_action=CATASRCH&p_onebar=&p_mode=AND&p_subj_cd=&p_subj=&p_dept=&p_school=&p_df=&p_submit='
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
# unmodified raw course categoriy html data
course_tag_soup = soup.find_all("td", class_="cataCourse")

# There are 6135 courses available for Spring 2021
number_courses = len(course_tag_soup)
#print(number_courses)

every_unique_course_tag = set()

for i in range(number_courses):
  #print(str(course_tag_soup[i])[-17:-13])
  every_unique_course_tag.add(str(course_tag_soup[i])[-17:-13])

# There are 182 different course categories
#print(len(every_unique_course_tag))

# sort course categories alphabetically
#print(sorted(every_unique_course_tag))

sorted_unqiue_course_categories = sorted(every_unique_course_tag)
#print(sorted_unqiue_course_categories)

In [4]:
class Course:

### Constructor methods
  def __init__(self, category, level, title, credit, url, distribution_group="NONE"):
    self.category = category
    self.level = level
    self.title = title
    self.distribution_group = distribution_group
    self.credit = credit
    self.url = url
    
### Accessor methods
  def get_category(self):
    return str(self.category)
  
  def get_level(self):
    return str(self.level)
  
  def get_title(self):
    return str(self.title)
  
  def get_distribution_group(self):
    return str(self.distribution_group)
  
  def get_credit(self):
    return str(self.credit)
  
  def get_url(self):
    return str(self.url)
    

Course Object Construction still uncomplete: I need to extract the prerequisite information out of every url, which requires some regular expression processing. Also I need to take consideration of every possible cases. Like a course A OR B. A AND B

After this I will need to work on the node UI generation, which basically requires a library to process the dependency graph and possible a good UI.

In [ ]:
url = "https://courses.rice.edu"+soup.tbody.find_all("tr")[863].find_all("td")[0].a["href"]
sub_page = requests.get(url)
sub_soup = BeautifulSoup(sub_page.content, 'html.parser')
#str(sub_soup)[str(sub_soup).find("Prerequisite(s):"):][str(sub_soup)[str(sub_soup).find("Prerequisite(s):"):].find(">")+1:str(sub_soup)[str(sub_soup).find("Prerequisite(s):"):].find("</div")-2]

str(sub_soup)[str(sub_soup).find("Recommended Prerequisite(s):"):][str(sub_soup)[str(sub_soup).find("Recommended Prerequisite(s):"):].find(": ")+2:str(sub_soup)[str(sub_soup).find("Recommended Prerequisite(s):"):].find(".")]

'(MATH 212 or MATH 222) and (CAAM 210 or COMP 140 or STAT 405) Mutually Exclusive: Cannot register for CAAM 334 if student has credit for CAAM 335'

In [5]:
tr_elements = soup.tbody.find_all("tr")

all_courses = []

for i in range(len(tr_elements)):
  tr_element = tr_elements[i]
  td_element = tr_element.td
  course_cate = td_element.get_text().split(" ")[0] # course category
  course_numb = td_element.get_text().split(" ")[1] # course level
  course_cred = str(tr_element.find_all("td", class_="credits"))[-7:-6] # course credit
  course_titl = tr_element.find_all("td", class_="cataTitle")[0].get_text() # course title
  course_dist = tr_element.find_all("td", class_="cataDist")[0].get_text() # course distribution

  ### Works on the url part and find dependent prerequisite
  parent = "https://courses.rice.edu"
  relative = tr_element.find_all("td")[0].a["href"]
  course_url = parent + relative

  sub_page = requests.get(course_url)
  sub_soup = BeautifulSoup(sub_page.content, 'html.parser')

  sub_soup_str = str(sub_soup)

  prer = sub_soup_str.find("Prerequisite(s)")
  if prer != -1:
    if sub_soup_str.find("Recommended Prerequisite(s)") != -1:
      prer = sub_soup_str.find("Recommended Prerequisite(s)")
      prer = sub_soup_str[prer:]
      start_pos = prer.find(": ") + 2
      if prer.find(".</") != -1:
        end_pos = prer.find(".</")
      else:
        end_pos = prer.find("</")
      prer = prer[start_pos:end_pos]
      if prer.find("Mutually Exclusive") == -1:
        print(course_cate + " " +  course_numb + " <R- " + prer)
      else:
        if prer.find(". Mutually") != -1:
          prer_r = prer[:prer.find(". Mutually")]
        else:
          prer_r = prer[:prer.find(" Mutually")]
        excl_s = prer[prer.find("t for")+6:]
        excl_s = excl_s[:-2]
        print(course_cate + " " + course_numb + " <R- " + prer_r)
        print(course_cate + " " + course_numb + " -X- " + excl_s)
    if sub_soup_str.find("<b>Prerequisite(s)") != -1:
      prer = sub_soup_str.find("Prerequisite(s)")
      prer = sub_soup_str[prer:]
      start_pos = prer.find(">") + 1
      end_pos = prer.find("</div") - 2
      prer = prer[start_pos:end_pos]
      print(course_cate + " " +  course_numb + " <- " + prer)
  
  pre_dep = []
    
  ###
  
  if course_dist == "":
    course = Course(course_cate, course_numb, course_titl, course_cred, course_url)
  else:
    course = Course(course_cate, course_numb, course_titl, course_cred, course_url, course_dist)
  #print(course_cate, course_numb, course_cred, course_titl, course_dist)
  all_courses.append(course)

#soup.tbody.find_all("tr")[0].td.get_text()
#soup.tbody.find_all("tr")[0].td.get_text().split(" ")
#str(soup.tbody.find_all("tr")[0].find_all("td", class_="credits"))[-7:-6]
#soup.tbody.find_all("tr")[0].find_all("td",class_="cataTitle")[0].get_text()
#soup.tbody.find_all("tr")[2].find_all("td",class_="cataDist")[0].get_text()
#soup.tbody.find_all("tr")[0].find_all("td",class_="credits")[0].get_text()

ANTH 300 <- ANTH 200 OR LING 200
ANTH 301 <- LING 200 OR ANTH 200
ANTH 303 <R- ANTH 205
ANTH 305 <- (ANTH 200 OR LING 200) AND (ANTH 301 OR LING 301)
ANTH 323 <- ANTH 200 OR LING 200
ANTH 362 <- ANTH 205
ANTH 399 <R- ANTH 381
ANTH 405 <R- ANTH 341
ANTH 407 <- (LING 300 OR ANTH 300) AND (LING 301 OR ANTH 301) AND (LING 304 OR ANTH 304) AND (LING 311 OR ANTH 323) OR (LING 500 OR ANTH 500) AND (LING 501 OR ANTH 501) AND (LING 504 OR ANTH 504) AND (LING 511 OR ANTH 523)
ANTH 408 <- ANTH 407 OR LING 407
ANTH 425 <- ANTH 205 AND ANTH 362
ANTH 446 <- ANTH 381
ANTH 460 <- ANTH 205
ANTH 505 <- ANTH 200 OR LING 200
ANTH 562 <- ANTH 205
ANTH 646 <R- ANTH 381 or ANTH 581
ANTH 646 -X- ANTH 446
ANTH 650 <R- Third year and above graduate students. Repeatable for Credit. 
ANTH 660 <- ANTH 205
ARAB 142 <- ARAB 141
ARAB 263 <- ARAB 142
ARAB 264 <- ARAB 263
ARAB 301 <- ARAB 264
ARAB 302 <- ARAB 301
ARCH 102 <- ARCH 101
ARCH 201 <- ARCH 102
ARCH 202 <- ARCH 201
ARCH 301 <- ARCH 202
ARCH 302 <- ARCH 301
AR